In [ ]:
# necessary modules

import numpy as np
from scipy.stats.qmc import LatinHypercube,discrepancy, scale
from monte_carlo_class import MonteCarloOptionPricing

In [ ]:
# LHS space creation for 9 variables with 200,000 samples (200000, 9)
sampler = LatinHypercube(9).random(n=200000)
print('LHS Space Size: ' + str(sampler.shape))

LHS Space Size: (200000, 9)


In [ ]:
# spot price, strike price, risk-free-rate, time-to-maturity, initial variance, volatility of volatility,
# long average variance, reversion speed, correlation
samples = scale(sampler, [200, 100, 0.02, 0.2, 0.05, 0, 0.2, 0.25, -0.9], [300, 400, 0.1, 1, 0.6, 0.3, 0.4, 1, 0])

# list to append prices to with their samples
samples_with_prices = []

# loop over hypercube samples and add their corresponding call and put prices to them
for i in range(len(samples+1)):
    
    MC = MonteCarloOptionPricing(S0 = round(samples[i][0], 4),
                                 K = round(samples[i][1], 4),
                                 T = round(samples[i][3], 4),
                                 r = round(samples[i][2], 4),
                                 sigma = round(samples[i][4], 4),
                                 div_yield = 0,
                                 simulation_rounds = 10000,
                                 no_of_slices = 40,
                                 fix_random_seed = False)
    
    MC.heston(kappa= round(samples[i][7], 4),
              theta = round(samples[i][6], 4),
              sigma_v = round(samples[i][5], 4),
              rho = round(samples[i][8], 4))
                                 
    MC.stock_price_simulation()
    
    
    call_price = MC.european_call()
    
    put_price = MC.european_put(call_price)
    
    updated_sample = [round(samples[i][0],4), round(samples[i][1],4),round(samples[i][2],4),
                    round(samples[i][3],4),round(samples[i][4],4),round(samples[i][5],4),
                    round(samples[i][6],4),round(samples[i][7],4),round(samples[i][8],4),
                    round(call_price,4), round(put_price, 4)]
    
    if i % 10000 == 0:
        print('Iteration: ' + str(i))
    
    samples_with_prices.append(updated_sample)

Iteration: 0
Iteration: 10000
Iteration: 20000
Iteration: 30000
Iteration: 40000
Iteration: 50000
Iteration: 60000
Iteration: 70000
Iteration: 80000
Iteration: 90000
Iteration: 100000
Iteration: 110000
Iteration: 120000
Iteration: 130000
Iteration: 140000
Iteration: 150000
Iteration: 160000
Iteration: 170000
Iteration: 180000
Iteration: 190000


In [ ]:
# test to see it worked

print(samples_with_prices[9000])

[242.9601, 243.4635, 0.0282, 0.4172, 0.3332, 0.2369, 0.2815, 0.3905, -0.0129, 17.6559, 15.3117]


In [ ]:
# saving
np.savetxt("heston_data", samples_with_prices, delimiter=",", )